In [1]:
from torch import nn
import torch
from torch.autograd import Variable
import sys
import shlex, subprocess
sys.path.append('/workspace/dense_cap')
sys.path.append('/workspace/dense_cap/densecap')
sys.path.append('/workspace/dense_cap/densecap/model')
sys.path.append('/workspace/dense_cap/tools/densevid_eval/coco-caption')
from tools.eval_proposal_anet import ANETproposal
from transformer import *
import torch
import numpy as np
import torch.nn.functional as F
from tqdm.auto import tqdm
from collections import defaultdict
import torch.optim as optim
from torch.optim import lr_scheduler
import math
from data.utils import segment_iou
import time
import os
import easydict
import random
from tqdm.auto import tqdm
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import DataLoader
from collections import defaultdict
import json
from tqdm.auto import tqdm
from torch.cuda.amp import autocast,GradScaler
from tqdm.notebook import tqdm
from tensorboardX import SummaryWriter
from data.anet_dataset_our_matching_deepct_video_prefixes_new_split import ANetDataset, anet_collate_fn, get_vocab_and_sentences
from data.anet_test_dataset_our_matching_deepct import ANetTestDataset, anet_test_collate_fn

In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["PATH"] = '/opt/conda/envs/py37/bin:/opt/conda/condabin:/opt/conda/bin:/opt/rh/devtoolset-7/root/usr/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/local/jdk-14.0.2/bin'
d_model =768
d_hidden = d_model*2
n_heads = 8
batch_size =12


In [3]:
evidence_name = "lxmert_sequence_contrast"
model_name = evidence_name + '_final'
start_from = "../checkpoint/{}/model_caption_best.t7".format(model_name)
start_from = ""

In [4]:
attention_weight = 0.1
temporal_weight = 0.1
multitask_weight = 0.1
learning_rate = 0.00001

In [5]:
args = easydict.EasyDict({
    'cfgs_file':'cfgs/our_yc2_only.yml', 'dataset':'our_yc2_only', 'dataset_file':'/workspace/dense_cap/densecap/data/our/our_yc2_only_annotations_new_trainval.json',
    'feature_root': "/workspace/dense_cap/our_dataset",'dur_file': "/workspace/dense_cap/densecap/data/our/our_yc2_only_duration_frame.csv", #'dist_url': "file:///workdir/mixiaoyue/nonexistent_file",
    'train_data_folder':['new_training'], 'val_data_folder': ['new_validation'],'test_data_folder':['new_testing'],'num_workers':0,
    'max_sentence_len':20, 'd_model':d_model, 'd_hidden': d_hidden, 'n_heads':n_heads, 'in_emb_dropout':0.1,
    'attn_dropout':0.2, 'vis_emb_dropout':0.1, 'cap_dropout':0.2, 'image_feat_size':3072, 'min_prop_num':50, 'max_prop_num':500,'min_prop_before_nms':200,
    'n_layers':2, 'train_sample':20, 'sample_prob':0, 'slide_window_size':560, 'slide_window_stride':20,
    'sampling_sec':1.5, 'kernel_list':[1, 2, 3, 4, 5, 7, 9, 11, 15, 21, 29, 41, 57, 71, 111, 161, 211, 251],
    'pos_thresh':0.7,'neg_thresh':0.3,'stride_factor':50, 'max_epochs':31,'batch_size':batch_size,'valid_batch_size':int(batch_size*1.5),
    'cls_weight':0.1, 'reg_weight':10, 'sent_weight':1.0, 'scst_weight':0.0, 'mask_weight':0.75, 'id':'{}'.format(model_name), 'optim':'adam',
    'learning_rate':learning_rate, 'alpha':0.95, 'beta':0.999, 'epsilon':1e-8, 'loss_alpha_r':2, 'patience_epoch':2, 'reduce_factor':0.9, 
    'grad_norm':1,'dist_backend':'gloo', 'world_size':1,'using_dual': False,'save_checkpoint_every':1,'checkpoint_path':'/workspace/dense_cap/checkpoint/models/lxmert_deepct',
    'losses_log_every':1,'seed':213,'cuda':'cuda','learn_mask':True,
    'save_train_samplelist':'store_true', 'load_train_samplelist': True, 'save_valid_samplelist':'store_true', 'load_valid_samplelist': True,
    'save_test_samplelist':'store_true', 'load_test_samplelist': True,'test_samplelist_path':'/workspace/dense_cap/samplelist/new_test_samplelist.pkl',
    'train_samplelist_path':'/workspace/dense_cap/samplelist/new_train_samplelist.pkl','valid_samplelist_path':'/workspace/dense_cap/samplelist/new_valid_samplelist.pkl',
    'test_raw_data':'/workspace/dense_cap/densecap/data/our/our_yc2_only_annotations_new_test.json',
    'start_from': start_from, 'distributed':False, 'using_recipe':True, 'using_dual':False, 'gated_mask':True, 'enable_visdom':False,
    'before':False, 'densecap_references': ["/workspace/dense_cap/densecap/data/our/new_test_our_yc2_only.json"],
    'densecap_eval_file': '/workspace/dense_cap/densecap/tools/densevid_eval/evaluate.py', 'epoch':50, 'query_transform_dim':d_model,
    'v_e_temporal':True,'v_v_contrast':False, 'v_e_contrast':True
})

In [6]:
torch.manual_seed(args.seed)
np.random.seed(args.seed)
random.seed(args.seed)

if args.cuda:
    torch.cuda.manual_seed_all(args.seed)

In [7]:
import json
with open('original_num_frame.json', 'r') as fp:
    of = json.load( fp)
    import pickle
newof = {}
for video_prefix, val in of.items():
    video_prefix = video_prefix.split('/')
    video_prefix[4] = 'new_testing'
    newof['/'.join(video_prefix)] = val
with open('duration_frame.pkl', 'rb') as fp:
    duration_frame = pickle.load( fp)
from scripts.train_our_copynet_adam_deepct import get_vocab_and_sentences
text_proc, raw_data = get_vocab_and_sentences(args.dataset_file, args.max_sentence_len)
gold_seg = {}
sampling_sec = 1.5
for vid, val in raw_data.items():
    gold_seg[vid] = {(int(item['segment'][0]/1.5),int(item['segment'][1]/1.5)):item['sentence']  for item in val['annotations']}


/opt/conda/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


dataset_file:  /workspace/dense_cap/densecap/data/our/our_yc2_only_annotations_new_trainval.json
# of words in the vocab: 4
# of sentences in training: 0, # of sentences in validation: 0
# of training videos: 0


In [8]:
import pickle as pkl
recipe_feat_path = os.path.join('/workspace', 'dense_cap', 'our_dataset', 'recipe_feat.pkl')
with open(recipe_feat_path, 'rb') as fp:
    recipe_feat = pkl.load(fp)
    
match_dict_path = os.path.join('/workspace/evidence_retrieval', evidence_name + '_matched_result.pkl')
with open(match_dict_path, 'rb') as fp:
    match_dict = pkl.load(fp)
def get_nearest_frame_idx(frame_idx, frame_indices):
    frame_indices = np.array(frame_indices)
    recipe_frame_index = np.argmin(np.abs(frame_indices-frame_idx))
    return frame_indices[recipe_frame_index]
def get_matched_recipe_feat(vids, duration_frame):
    batch_recipe_feat = []
    for b, vid in enumerate(vids):
        vid_match = match_dict[vid]
        vid_recipe_key = list(vid_match.keys())
        vid_recipe_key.sort()
        vid_recipe_feat = []
        for i in range(560):
            if i <duration_frame[vid]:
                matched_recipe_index = get_nearest_frame_idx(i, vid_recipe_key)
                if matched_recipe_index not in vid_match.keys():
                    matched_recipe_index = max(vid_match.keys())
                frame_recipe_feat = recipe_feat['split_ins'][vid_match[matched_recipe_index][0]]
            else:
                frame_recipe_feat = torch.zeros_like(frame_recipe_feat)
            vid_recipe_feat.append(frame_recipe_feat)
        vid_recipe_feat = torch.cat(vid_recipe_feat).unsqueeze(0)
        batch_recipe_feat.append(vid_recipe_feat)
    batch_recipe_feat = torch.cat(batch_recipe_feat)
    return batch_recipe_feat

In [9]:
match_dict_path

'/workspace/evidence_retrieval/lxmert_sequence_contrast_matched_result.pkl'

In [10]:
def positional_encodings(x, D):
    # input x a vector of positions
    encodings = torch.zeros(x.size(0), D)
    if x.is_cuda:
        encodings = encodings.cuda(x.get_device())
    encodings = Variable(encodings)

    for channel in range(D):
        if channel % 2 == 0:
            encodings[:,channel] = torch.sin(
                x / 10000 ** (channel / D))
        else:
            encodings[:,channel] = torch.cos(
                x / 10000 ** ((channel - 1) / D))
    return encodings

In [11]:


class DropoutTime1D(nn.Module):
    '''
        assumes the first dimension is batch, 
        input in shape B x T x H
        '''
    def __init__(self, p_drop):
        super(DropoutTime1D, self).__init__()
        self.p_drop = p_drop

    def forward(self, x):
        if self.training:
            mask = x.data.new(x.data.size(0),x.data.size(1), 1).uniform_()
            mask = Variable((mask > self.p_drop).float())
            
            return x * mask
        else:
            return x * (1-self.p_drop)

    def init_params(self):
        pass

    def __repr__(self):
        repstr = self.__class__.__name__ + ' (\n'
        repstr += "{:.2f}".format(self.p_drop)
        repstr += ')'
        return repstr

In [12]:
class ActionPropDenseCap(nn.Module):
    def __init__(self, d_model, d_hidden, n_layers, n_heads, vocab,
                 in_emb_dropout, attn_dropout, vis_emb_dropout,
                 cap_dropout, nsamples, kernel_list, stride_factor,
                 learn_mask=False, window_length=560):
        super(ActionPropDenseCap, self).__init__()
        print("voca len: ", len(vocab))
        self.kernel_list = kernel_list
        self.nsamples = nsamples
        self.learn_mask = learn_mask
        self.d_model = d_model
        self.dim_embedder = nn.Linear(768*2,d_model)
        self.mask_model = nn.Sequential(
            nn.Linear(d_model+window_length, d_model, bias=False),
            nn.BatchNorm1d(d_model),
            nn.ReLU(),
            nn.Linear(d_model, window_length),
            #nn.Sigmoid()
        )
        
        #self.query_embed = segment_query_embed(args)
        self.vis_emb = Transformer(d_model, 0, 0,
                                   d_hidden=d_hidden,
                                   n_layers=n_layers,
                                   n_heads=n_heads,
                                   drop_ratio=attn_dropout)

        self.vis_dropout = DropoutTime1D(vis_emb_dropout)
        #args.query_transform_dim
        self.prop_out = nn.ModuleList(
            [nn.Sequential(
                nn.BatchNorm1d(d_model),
                nn.Conv1d(d_model, d_model,
                          self.kernel_list[i],
                          stride=math.ceil(kernel_list[i]/stride_factor),
                          groups=d_model,
                          bias=False),
                nn.BatchNorm1d(d_model),
                nn.Conv1d(d_model, d_model,
                          1, bias=False),
                nn.BatchNorm1d(d_model),
                nn.ReLU(inplace=True),
                nn.BatchNorm1d(d_model),
                nn.Conv1d(d_model, 4,
                          1)
                )
                for i in range(len(self.kernel_list))
            ])#+args.query_transform_dim

        #print("d_model: ", d_model)
        self.cap_model = RealTransformer(d_model,
                                         self.vis_emb.encoder, #share the encoder
                                         vocab,
                                         d_hidden=d_hidden,
                                         n_layers=n_layers,
                                         n_heads=n_heads,
                                         drop_ratio=cap_dropout)

        self.bce_loss = nn.BCEWithLogitsLoss()
        self.reg_loss = nn.SmoothL1Loss()
        self.l2_loss = nn.MSELoss()
        
    def forward(self, x, s_pos, s_neg, sentence,
                sample_prob=0, stride_factor=10, scst=False,
                gated_mask=False, using_recipe=False, recipe_vector = None, dual=False, nff = None,vids = None, gold_segs = None):
        B, T, _ = x.size()
        dtype = x.data.type()

        #x_rgb, x_flow = torch.split(x, 2048, 2)
        #x_rgb = self.rgb_emb(x_rgb.contiguous())
        #x_flow = self.flow_emb(x_flow.contiguous())

        
        #x = torch.cat((x_rgb, x_flow), 2)
        
        #print("x: ", x.shape)
        #print("after x: ", x.shape)
        if using_recipe and recipe_vector is not None:
            #query_with_attention, temporal_loss, attention_score  = self.query_embed(recipe_vector, _final_tensor = vis_feat,
            #                                                                         vids= vids, gold_segs = gold_segs)
            x = torch.cat([x, recipe_vector], dim = -1)
        x = self.dim_embedder(x)
        vis_feat, all_emb = self.vis_emb(x)#, query=False, dual=False)
        #print('visfeat',vis_feat.shape)
        """if using_recipe and recipe_vector is not None:
            query_with_attention, temporal_loss, attention_score  = self.query_embed(recipe_vector, _final_tensor = vis_feat,
                                                                                     vids= vids, gold_segs = gold_segs)
            vis_feat = torch.cat([vis_feat, query_with_attention], dim = -1)"""

        # B x T x H -> B x H x T
        # for 1d conv
        vis_feat = vis_feat.transpose(1,2).contiguous()

        prop_lst = []
        for i, kernel in enumerate(self.prop_out):

            kernel_size = self.kernel_list[i]
            if kernel_size <= vis_feat.size(-1):
                pred_o = kernel(vis_feat)
                anchor_c = Variable(torch.FloatTensor(np.arange(
                    float(kernel_size)/2.0,
                    float(T+1-kernel_size/2.0),
                    math.ceil(kernel_size/stride_factor)
                )).type(dtype))
                if anchor_c.size(0) != pred_o.size(-1):
                    raise Exception("size mismatch!")

                anchor_c = anchor_c.expand(B, 1, anchor_c.size(0))
                anchor_l = Variable(torch.FloatTensor(anchor_c.size()).fill_(kernel_size).type(dtype))

                pred_final = torch.cat((pred_o, anchor_l, anchor_c), 1)
                
                prop_lst.append(pred_final)
            else:
                #print('skipping kernel sizes greater than {}'.format(self.kernel_list[i]))
                break

        # Important! In prop_all, for the first dimension, the four values are proposal score, overlapping score (DEPRECATED!), length offset, and center offset, respectively
        prop_all = torch.cat(prop_lst, 2)
        #print("prop_all: ", prop_all.shape)
        if B != s_pos.size(0) or B != s_neg.size(0):
            raise Exception('feature and ground-truth segment do not match!')

        sample_each = self.nsamples // 2
        pred_score = Variable(torch.FloatTensor(np.zeros((sample_each*B, 2))).type(dtype))
        gt_score = Variable(torch.FloatTensor(np.zeros((sample_each*B, 2))).type(dtype))
        pred_offsets = Variable(torch.FloatTensor(np.zeros((sample_each*B,2))).type(dtype))
        gt_offsets = Variable(torch.FloatTensor(np.zeros((sample_each*B,2))).type(dtype))

        # B x T x H
        batch_mask = Variable(torch.FloatTensor(np.zeros((B,T,1))).type(dtype))

        # store positional encodings, size of B x 4,
        # the first B values are predicted starts,
        # second B values are predicted ends,
        # third B values are anchor starts,
        # last B values are anchor ends
        pe_locs = Variable(torch.zeros(B*4).type(dtype))
        anchor_window_mask = Variable(torch.zeros(B, T).type(dtype),
                                      requires_grad=False)
        pred_bin_window_mask = Variable(torch.zeros(B, T).type(dtype),
                                      requires_grad=False)
        gate_scores = Variable(torch.zeros(B,1,1).type(dtype))

        mask_loss = None

        pred_len = prop_all[:, 4, :] * torch.exp(prop_all[:, 2, :])
        pred_cen = prop_all[:, 5, :] + prop_all[:, 4, :] * prop_all[:, 3, :]

        for b in range(B):
            pos_anchor = s_pos[b]
            neg_anchor = s_neg[b]
            #print("pos_anchor size: ", pos_anchor.size(0))
            #print('neg_anchor size: ', neg_anchor.size(0))

            if pos_anchor.size(0) != sample_each or neg_anchor.size(0) != sample_each:
                raise Exception("# of positive or negative samples does not match")

            # randomly choose one of the positive samples to caption
            pred_index = np.random.randint(sample_each)

            # random sample anchors from different length
            for i in range(sample_each):
                # sample pos anchors
                pos_sam = pos_anchor[i].data
                pos_sam_ind = int(pos_sam[0])

                pred_score[b*sample_each+i, 0] = prop_all[b, 0, pos_sam_ind]
                gt_score[b*sample_each+i, 0] = 1

                pred_offsets[b*sample_each+i] = prop_all[b, 2:4, pos_sam_ind]
                gt_offsets[b*sample_each+i] = pos_sam[2:]

                # sample neg anchors
                neg_sam = neg_anchor[i].data
                neg_sam_ind = int(neg_sam[0])
                pred_score[b*sample_each+i, 1] = prop_all[b, 0, neg_sam_ind]
                gt_score[b*sample_each+i, 1] = 0

                # caption the segment
                if i == pred_index: # only need once, since one sample corresponds to one sentence only
                           # TODO Is that true? Why cannot caption all?
                    # anchor length, 4, 5 are the indices for anchor length and center
                    anc_len = prop_all[b, 4, pos_sam_ind].data.item()
                    anc_cen = prop_all[b, 5, pos_sam_ind].data.item()

                    # grount truth length, 2 and 3 are indices for ground truth length and center
                    # see line 260 and 268 in anet_dataset.py
                    # dont need to use index since now i is 0 and everything is matching
                    # length is after taking log
                    gt_len = np.exp(pos_sam[2].item()) * anc_len
                    gt_cen = pos_sam[3].item() * anc_len + anc_cen
                    gt_window_mask = torch.zeros(T, 1).type(dtype)

                    gt_window_mask[
                    max(0, math.floor(gt_cen - gt_len / 2.)):
                    min(T, math.ceil(gt_cen + gt_len / 2.)), :] = 1.
                    gt_window_mask = Variable(gt_window_mask,
                                              requires_grad=False)

                    batch_mask[b] = gt_window_mask
                    # batch_mask[b] = anchor_window_mask

                    crt_pred_cen = pred_cen[b, pos_sam_ind]
                    crt_pred_len = pred_len[b, pos_sam_ind]
                    pred_start_w = crt_pred_cen - crt_pred_len / 2.0
                    pred_end_w = crt_pred_cen + crt_pred_len/2.0

                    pred_bin_window_mask[b,
                    math.floor(max(0, min(T-1, pred_start_w.data.item()))):
                    math.ceil(max(1, min(T, pred_end_w.data.item())))] = 1.

                    if self.learn_mask:
                        anchor_window_mask[b,
                        max(0, math.floor(anc_cen - anc_len / 2.)):
                        min(T, math.ceil(anc_cen + anc_len / 2.))] = 1.

                        pe_locs[b] = pred_start_w
                        pe_locs[B + b] = pred_end_w
                        pe_locs[B*2 + b] = Variable(torch.Tensor([max(0, math.floor(anc_cen - anc_len / 2.))]).type(dtype))
                        pe_locs[B*3 + b] = Variable(torch.Tensor([min(T, math.ceil(anc_cen + anc_len / 2.))]).type(dtype))

                        # gate_scores[b] = pred_score[b*sample_each+i, 0].detach()
                        gate_scores[b] = pred_score[b*sample_each+i, 0]

        #print("self.learn_mask: ", self.learn_mask)
        if self.learn_mask:
            pos_encs = positional_encodings(pe_locs, self.d_model//4)
            in_pred_mask = torch.cat((pos_encs[:B], pos_encs[B:B*2],
                                      pos_encs[B*2:B*3], pos_encs[B*3:B*4],
                                      anchor_window_mask), 1)
            pred_mask = self.mask_model(in_pred_mask).view(B, T, 1)

            #print("gated_mask: ", gated_mask)
            if gated_mask:
                gate_scores = F.sigmoid(gate_scores)
                window_mask = (gate_scores * pred_bin_window_mask.view(B, T, 1)
                # window_mask = (gate_scores * batch_mask
                                +  (1-gate_scores) * pred_mask)
            else:
                window_mask = pred_mask

                
            # Try to remove nan loss error
            pred_mask.clamp (min=-10, max=10)
            pred_bin_window_mask.clamp (min=-10, max=10)

            mask_loss = F.binary_cross_entropy_with_logits(pred_mask + 1e-10, pred_bin_window_mask.view(B, T, 1) + 1e-10)
            # mask_loss = F.binary_cross_entropy_with_logits(window_mask, batch_mask)
        else:
            window_mask = pred_bin_window_mask.view(B, T, 1)
            # window_mask = batch_mask

        #print("x: ", x.shape)
        #print("sentence: ", sentence.shape, sentence)
        #print("window mask: ", window_mask.shape)
        #print("*" * 50)
        pred_sentence, gt_cent, in_encoding, h_from_decoder = self.cap_model(x, sentence, window_mask,
                                               sample_prob=sample_prob)

        scst_loss = None
        #print("scst: ", scst)
        if scst:
            scst_loss = self.cap_model.scst(x, batch_mask, sentence)
            
        pred_sentence = pred_sentence
            

            
        return (pred_score, gt_score,
                pred_offsets, gt_offsets,
                pred_sentence, gt_cent,
                scst_loss, mask_loss, h_from_decoder)#, temporal_loss, attention_score)


    def inference(self, x, actual_frame_length, sampling_sec,
                  min_prop_num, max_prop_num,
                  min_prop_num_before_nms, pos_thresh, stride_factor,
                  gated_mask=False, using_recipe=False, recipe_vector = None, dual = False):
        B, T, _ = x.size()
        dtype = x.data.type()

        #x_rgb, x_flow = torch.split(x, 2048, 2)
        #x_rgb = self.rgb_emb(x_rgb.contiguous())
        #x_flow = self.flow_emb(x_flow.contiguous())

        #x = torch.cat((x_rgb, x_flow), 2)
            
        
        if using_recipe and recipe_vector is not None:
            #query_with_attention, temporal_loss, attention_score  = self.query_embed(recipe_vector, _final_tensor = vis_feat,
            #                                                                         vids= vids, gold_segs = gold_segs)
            x = torch.cat([x, recipe_vector], dim = -1)
        x = self.dim_embedder(x)
        vis_feat, all_emb = self.vis_emb(x)#, query=False, dual=False)
        vis_feat = vis_feat.transpose(1,2).contiguous()

        prop_lst = []
        for i, kernel in enumerate(self.prop_out):

            kernel_size = self.kernel_list[i]
            if kernel_size <= actual_frame_length[0]: # no need to use larger kernel size in this case, batch size is only 1
                pred_o = kernel(vis_feat)
                anchor_c = Variable(torch.FloatTensor(np.arange(
                    float(kernel_size)/2.0,
                    float(T+1-kernel_size/2.0),
                    math.ceil(kernel_size/stride_factor)
                )).type(dtype))
                if anchor_c.size(0) != pred_o.size(-1):
                    raise Exception("size mismatch!")

                anchor_c = anchor_c.expand(B, 1, anchor_c.size(0))
                anchor_l = Variable(torch.FloatTensor(anchor_c.size()).fill_(kernel_size).type(dtype).cuda())

                pred_final = torch.cat((pred_o, anchor_l, anchor_c), 1)
                prop_lst.append(pred_final)

        prop_all = torch.cat(prop_lst, 2)

        # assume 1st and 2nd are action prediction and overlap, respectively
        prop_all[:,:2,:] = F.sigmoid(prop_all[:,:2,:])


        pred_len = prop_all[:, 4, :] * torch.exp(prop_all[:, 2, :])
        pred_cen = prop_all[:, 5, :] + prop_all[:, 4, :] * prop_all[:, 3, :]

        nms_thresh_set = np.arange(0.9, 0.95, 0.05).tolist()
        all_proposal_results = []

        # store positional encodings, size of B x 4,
        # the first B values are predicted starts,
        # second B values are predicted ends,
        # third B values are anchor starts,
        # last B values are anchor ends
        
        for b in range(B):
            crt_pred = prop_all.data[b]
            crt_pred_cen = pred_cen.data[b]
            crt_pred_len = pred_len.data[b]
            pred_start_lst = [] #torch.zeros(B * 4).type(dtype)
            pred_end_lst = []
            anchor_start_lst = []
            anchor_end_lst = []

            pred_masks = []
            batch_result = []
            anchor_window_mask = [] #Variable(torch.zeros(B, T).type(dtype))
            gate_scores = [] #Variable(torch.zeros(B, 1).type(dtype))
            crt_nproposal = 0
            nproposal = torch.sum(torch.gt(prop_all.data[b, 0, :], pos_thresh))
            nproposal = min(max(nproposal, min_prop_num_before_nms),
                            prop_all.size(-1))
            pred_results = np.empty((nproposal, 3))
            _, sel_idx = torch.topk(crt_pred[0], nproposal)
 
            start_t = time.time()
            for nms_thresh in nms_thresh_set:
                for prop_idx in range(nproposal):
                    #print("AAAAAAAAAAAA")
                    original_frame_len = actual_frame_length[b].item() + sampling_sec*2 # might be truncated at the end, hence + frame_to_second*2
                    pred_start_w = crt_pred_cen[sel_idx[prop_idx]] - crt_pred_len[sel_idx[prop_idx]] / 2.0
                    #print("pred_start_w: ", pred_start_w)
                    pred_end_w = crt_pred_cen[sel_idx[prop_idx]] + crt_pred_len[sel_idx[prop_idx]] / 2.0
                    pred_start = pred_start_w
                    pred_end = pred_end_w
                    if pred_start >= pred_end:
                        continue
                    if pred_end >= original_frame_len or pred_start < 0:
                        continue

                    hasoverlap = False
                    if crt_nproposal > 0:
                        if np.max(segment_iou(np.array([pred_start.cpu(), pred_end.cpu()]), pred_results[:crt_nproposal])) > nms_thresh:
                            hasoverlap = True

                    if not hasoverlap:
                        pred_bin_window_mask = torch.zeros(1, T, 1).type(dtype)
                        win_start = math.floor(max(min(pred_start, min(original_frame_len, T)-1), 0))
                        win_end = math.ceil(max(min(pred_end, min(original_frame_len, T)), 1))
                        # if win_start >= win_end:
                        #     print('length: {}, mask window start: {} >= window end: {}, skipping'.format(
                        #         original_frame_len, win_start, win_end,
                        #     ))
                        #     continue

                        pred_bin_window_mask[:, win_start:win_end] = 1
                        pred_masks.append(pred_bin_window_mask)

                        if self.learn_mask:
                            # 4, 5 are the indices for anchor length and center
                            anc_len = crt_pred[4, sel_idx[prop_idx]]
                            anc_cen = crt_pred[5, sel_idx[prop_idx]]
                            # only use the pos sample to train, could potentially use more sample for training mask, but this is easier to do
                            amask = torch.zeros(1,T).type(dtype)
                            amask[0,
                            max(0, math.floor(anc_cen - anc_len / 2.)):
                            min(T, math.ceil(anc_cen + anc_len / 2.))] = 1.
                            anchor_window_mask.append(amask)

                            pred_start_lst.append(torch.Tensor([pred_start_w]).type(dtype).cuda())
                            #print("pred_start_lst: ", pred_start_lst)
                            pred_end_lst.append(torch.Tensor([pred_end_w]).type(dtype).cuda())
                            anchor_start_lst.append(torch.Tensor([max(0,
                                                                 math.floor(
                                                                 anc_cen - anc_len / 2.))]).type(
                                                                 dtype).cuda())
                            anchor_end_lst.append(torch.Tensor([min(T,
                                                               math.ceil(
                                                               anc_cen + anc_len / 2.))]).type(
                                                               dtype).cuda())

                            gate_scores.append(torch.Tensor([crt_pred[0, sel_idx[prop_idx]]]).type(dtype).cuda())

                        pred_results[crt_nproposal] = np.array([win_start,
                                                                win_end,
                                                                crt_pred[0, sel_idx[prop_idx]]])
                        crt_nproposal += 1

                    if crt_nproposal >= max_prop_num:
                        break

                if crt_nproposal >= min_prop_num:
                    break

            mid1_t = time.time()

            if len(pred_masks) == 0: # append all-one window if no window is proposed
                pred_masks.append(torch.ones(1, T, 1).type(dtype))
                pred_results[0] = np.array([0, min(original_frame_len, T), pos_thresh])
                crt_nproposal = 1

            pred_masks = Variable(torch.cat(pred_masks, 0))
            batch_x = x[b].unsqueeze(0).expand(pred_masks.size(0), x.size(1), x.size(2))

            if self.learn_mask:
                pe_pred_start = torch.cat(pred_start_lst, 0)
                pe_pred_end = torch.cat(pred_end_lst, 0)
                pe_anchor_start = torch.cat(anchor_start_lst, 0)
                pe_anchor_end = torch.cat(anchor_end_lst, 0)

                pe_locs = torch.cat((pe_pred_start, pe_pred_end, pe_anchor_start, pe_anchor_end), 0)
                pos_encs = positional_encodings(pe_locs, self.d_model // 4)
                npos = pos_encs.size(0)
                anchor_window_mask = Variable(torch.cat(anchor_window_mask,dim =  0))
                in_pred_mask = torch.cat((pos_encs[:npos//4], pos_encs[npos//4:npos//4*2],
                                          pos_encs[npos//4 * 2:npos//4 * 3],
                                          pos_encs[npos//4 * 3:npos//4 * 4],
                                          anchor_window_mask), dim = 1)
                pred_cont_masks  = self.mask_model(in_pred_mask).unsqueeze(2)

                if gated_mask:
                    gate_scores = Variable(torch.cat(gate_scores, 0).view(-1,1,1))
                    window_mask = (gate_scores * pred_masks
                                   + (1 - gate_scores) * pred_cont_masks)

                else:
                    window_mask = pred_cont_masks
            else:
                window_mask = pred_masks

            mid2_t = time.time()

            
            pred_sentence = []
            # use cap_batch as caption batch size
            cap_batch = math.ceil(560*256/T)
            for sent_i in range(math.ceil(window_mask.size(0)/cap_batch)):
                batch_start = sent_i*cap_batch
                batch_end = min((sent_i+1)*cap_batch, window_mask.size(0))
                temp_len = batch_x[batch_start:batch_end].shape[0]
                pred_sentence += self.cap_model.greedy(batch_x[batch_start:batch_end],
                                                       window_mask[batch_start:batch_end],20)

            pred_results = pred_results[:crt_nproposal]
            assert len(pred_sentence) == crt_nproposal, (
                "number of predicted sentence and proposal does not match"
            )

            for idx in range(len(pred_results)):
                batch_result.append((pred_results[idx][0],
                                     pred_results[idx][1],
                                     pred_results[idx][2],
                                     pred_sentence[idx]))
            all_proposal_results.append(tuple(batch_result))

            end_t = time.time()
            #print('Processing time for tIoU: {:.2f}, mask: {:.2f}, caption: {:.2f}'.format(mid1_t-start_t, mid2_t-mid1_t, end_t-mid2_t))

        return all_proposal_results

In [13]:
def get_dataset(args):
    print("Process Dataset")
    print(args.dataset_file)
    # process text
    text_proc, raw_data = get_vocab_and_sentences(args.dataset_file, args.max_sentence_len)
    print("Create Dataset...")
    print("feature root: ", args.feature_root)
    # Create the dataset and data loader instance
    if 'deepct' in model_name:
        recipt_pkl = "/workspace/Dataset/lxmert_deepct_matching_recipe.pkl"
        recipt2bert_pkl = "/workspace/Dataset/lxmert_deepct_matching_recipe2bert_onesent.pkl"
    elif 'match' in model_name:
        recipt_pkl = "/workspace/Dataset/lxmert_matching_recipe.pkl"
        recipt2bert_pkl = "/workspace/Dataset/lxmert_matching_recipe2bert_onesent.pkl"
    else :
        recipt_pkl = "/workspace/Dataset/bm25_matching_recipe.pkl"
        recipt2bert_pkl = "/workspace/Dataset/bm25_matching_recipe2bert_onesent.pkl"
    train_dataset = ANetDataset(args.feature_root,
                                args.train_data_folder,
                                args.slide_window_size,
                                args.dur_file,
                                args.kernel_list,
                                text_proc, raw_data,
                                args.pos_thresh, args.neg_thresh,
                                args.stride_factor,
                                args.dataset,
                                save_samplelist=args.save_train_samplelist,
                                load_samplelist=args.load_train_samplelist,
                                sample_listpath=args.train_samplelist_path,
                                recipt_pkl = recipt_pkl, recipt2bert_pkl = recipt2bert_pkl
                                )

    # dist parallel, optional
    args.distributed = args.world_size > 1
    if args.distributed and args.cuda:
        dist.init_process_group(backend=args.dist_backend, init_method=args.dist_url,
                                world_size=args.world_size)
        train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
    else:
        train_sampler = None

    train_loader = DataLoader(train_dataset,
                              batch_size=args.batch_size,
                              shuffle=(train_sampler is None), sampler=train_sampler,
                              num_workers=args.num_workers,
                              collate_fn=anet_collate_fn, drop_last = True)

    valid_dataset = ANetDataset(args.feature_root,
                                args.val_data_folder,
                                args.slide_window_size,
                                args.dur_file,
                                args.kernel_list,
                                text_proc, raw_data,
                                args.pos_thresh, args.neg_thresh,
                                args.stride_factor,
                                args.dataset,
                                save_samplelist=args.save_valid_samplelist,
                                load_samplelist=args.load_valid_samplelist,
                                sample_listpath=args.valid_samplelist_path,
                                recipt_pkl = recipt_pkl, recipt2bert_pkl = recipt2bert_pkl
                                )

    valid_loader = DataLoader(valid_dataset,
                              batch_size=args.valid_batch_size,
                              shuffle=False,
                              num_workers=args.num_workers,
                              collate_fn=anet_collate_fn)

    return train_loader, valid_loader, text_proc, train_sampler

def get_model(text_proc, args):
    sent_vocab = text_proc.vocab
    model = ActionPropDenseCap(d_model=args.d_model,
                               d_hidden=args.d_hidden,
                               n_layers=args.n_layers,
                               n_heads=args.n_heads,
                               vocab=sent_vocab,
                               in_emb_dropout=args.in_emb_dropout,
                               attn_dropout=args.attn_dropout,
                               vis_emb_dropout=args.vis_emb_dropout,
                               cap_dropout=args.cap_dropout,
                               nsamples=args.train_sample,
                               kernel_list=args.kernel_list,
                               stride_factor=args.stride_factor,
                               learn_mask=args.mask_weight>0, window_length=args.slide_window_size)

    # Initialize the networks and the criterion
    if len(args.start_from) > 0:
        print("Initializing weights from {}".format(args.start_from))
        model.load_state_dict(torch.load(args.start_from,
                                              map_location=lambda storage, location: storage))

    # Ship the model to GPU, maybe
    if args.cuda:
        model.cuda()
        if args.distributed:
            model = torch.nn.parallel.DistributedDataParallel(model)
        # elif torch.cuda.device_count() > 1:
        #     model = torch.nn.DataParallel(model).cuda()
        # else:
        #     model.cuda()
    return model

In [14]:
from scripts.train_our_copynet_adam_deepct_new_split import get_dataset

train_loader, valid_loader, test_loader,text_proc, train_sampler = get_dataset(args)

Process Dataset
/workspace/dense_cap/densecap/data/our/our_yc2_only_annotations_new_trainval.json


/opt/conda/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


dataset_file:  /workspace/dense_cap/densecap/data/our/our_yc2_only_annotations_new_trainval.json
# of words in the vocab: 841
# of sentences in training: 7063, # of sentences in validation: 2472
# of training videos: 910
Create Dataset...
feature root:  /workspace/dense_cap/our_dataset
image_path:  /workspace/dense_cap/our_dataset
split:  ['new_training']
split paths:  ['/workspace/dense_cap/our_dataset/new_training']
Load recipe dataset...
image_path:  /workspace/dense_cap/our_dataset
split:  ['new_validation']
split paths:  ['/workspace/dense_cap/our_dataset/new_validation']
Load recipe dataset...
image_path:  /workspace/dense_cap/our_dataset
split:  ['new_testing']
split paths:  ['/workspace/dense_cap/our_dataset/new_testing']
Load recipe dataset...


In [15]:
len(train_loader.dataset.sample_list), len(valid_loader.dataset.sample_list), len(test_loader.dataset.sample_list)

(6994, 2440, 1057)

In [16]:
model = get_model(text_proc, args)

voca len:  841


In [17]:
from datetime import datetime
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
writer = SummaryWriter('checkpoint/{}/{}'.format(model_name, timestamp))
print(timestamp)

2022-09-14 04:14:28


In [18]:
#changed adam to adamw

optimizer = optim.AdamW(
            filter(lambda p: p.requires_grad, model.parameters()),
            args.learning_rate, betas=(args.alpha, args.beta), eps=args.epsilon)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, factor=args.reduce_factor,
                                               patience=args.patience_epoch,
                                               verbose=True)

In [19]:
best_loss = 1000
vis, vis_window = None, None
for epoch in tqdm(range(args.epoch)):
    model.train() # training mode
    train_loss = []
    nbatches = len(train_loader)
    t_iter_start = time.time()

    sample_prob = min(args.sample_prob, int(epoch/5)*0.05)
    scaler = GradScaler()
    with autocast():
        for train_iter, data in enumerate(train_loader):
            (img_batch, tempo_seg_pos, tempo_seg_neg, sentence_batch, query, _,recipe_vector, neg_frame_flag, video_prefixes) = data
            img_batch = Variable(img_batch)
            tempo_seg_pos = Variable(tempo_seg_pos)
            tempo_seg_neg = Variable(tempo_seg_neg)
            sentence_batch = Variable(sentence_batch)

            vids = [prefix.split('/')[-1] for prefix in video_prefixes]
            bert_split = get_matched_recipe_feat(vids, duration_frame)
            if args.cuda:
                img_batch = img_batch.cuda()
                tempo_seg_neg = tempo_seg_neg.cuda()
                tempo_seg_pos = tempo_seg_pos.cuda()
                sentence_batch = sentence_batch.cuda()
                recipe_vector = recipe_vector.cuda()
                neg_frame_flag =  neg_frame_flag.cuda()
                bert_split = bert_split.cuda()
            t_model_start = time.time()
            batch_gold_seg = [list(gold_seg[vid].keys()) for vid in vids]
            (pred_score, gt_score,
            pred_offsets, gt_offsets,
            pred_sentence, gt_sent,
             scst_loss, mask_loss,_) = model(img_batch, tempo_seg_pos,
                                           tempo_seg_neg, sentence_batch,
                                           sample_prob, args.stride_factor,
                                           scst=args.scst_weight > 0,
                                           gated_mask=args.gated_mask, 
                                          using_recipe = args.using_recipe, recipe_vector = bert_split,
                                            vids = vids, gold_segs = batch_gold_seg)
            #gold_attention = build_gold_attention(batch_gold_seg)
            #gold_attention = gold_attention.to(attention_score.device)
            #attention_loss =  F.binary_cross_entropy_with_logits(attention_score + 1e-10, gold_attention+ 1e-10)
            cls_loss = model.bce_loss(pred_score + 1e-10, gt_score + 1e-10) * args.cls_weight
            reg_loss = model.reg_loss(pred_offsets + 1e-10, gt_offsets + 1e-10) * args.reg_weight

            #print("pred_sentence", pred_sentence, type(pred_sentence))
            #print("gt_sent", gt_sent, type(gt_sent))
            ################################
            ### Add dynamic sent_weight
            ###############################
            min_sent_weight = 0.5
            min_sent_epoch = 15

            #sent_loss = F.cross_entropy(pred_sentence + 1e-10 , gt_sent ) * args.sent_weight
            tmp_sent_weight = args.sent_weight - (args.sent_weight - min_sent_weight) * min(1.0, epoch/min_sent_epoch)
            sent_loss = F.cross_entropy(pred_sentence + 1e-10 , gt_sent ) * tmp_sent_weight
            total_loss = cls_loss + reg_loss + sent_loss #+ attention_loss*attention_weight+ temporal_loss*temporal_weight

            ################################
            ### Add dynamic mask_weight
            ###############################
            min_mask_weight = 0.1
            min_mask_epoch = 15

            if scst_loss is not None:
                scst_loss *= args.scst_weight
                total_loss += scst_loss

            if mask_loss is not None:
                #mask_loss = args.mask_weight * mask_loss
                mask_loss = (args.mask_weight - (args.mask_weight - min_mask_weight) * min(1.0, epoch / min_mask_epoch)) * mask_loss
                total_loss += mask_loss
            else:
                mask_loss = cls_loss.new(1).fill_(0)
            writer.add_scalar('train/cls_loss', cls_loss.item(),epoch*len(train_loader)+train_iter)
            writer.add_scalar('train/sent_loss', sent_loss.item(),epoch*len(train_loader)+train_iter)
            writer.add_scalar('train/reg_loss', reg_loss.item(),epoch*len(train_loader)+train_iter)
            writer.add_scalar('train/total_loss', total_loss.item(),epoch*len(train_loader)+train_iter)
            scaler.scale(total_loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            train_loss.append(total_loss.data.item())

        t_model_end = time.time()
        print('iter: [{}/{}], training loss: {:.4f}, '
              'class: {:.4f}, '
              'reg: {:.4f}, sentence: {:.4f}, '
              'mask: {:.4f}, '
              'data time: {:.4f}s, total time: {:.4f}s'.format(
            train_iter, nbatches, total_loss.data.item(), cls_loss.data.item(),
            reg_loss.data.item(), sent_loss.data.item(), mask_loss.data.item(),
            t_model_start - t_iter_start,
            t_model_end - t_iter_start
        ), end='\r')

        t_iter_start = time.time()
        valid_loss=[]
        val_cls_loss=[]
        val_reg_loss=[]
        val_sent_loss=[]
        val_mask_loss=[]
        for iter_num, data in enumerate(valid_loader):
            (img_batch, tempo_seg_pos, tempo_seg_neg, sentence_batch, query, _, recipe_vector, neg_frame_flag, video_prefixes) = data
            with torch.no_grad():
                img_batch = Variable(img_batch)
                tempo_seg_pos = Variable(tempo_seg_pos)
                tempo_seg_neg = Variable(tempo_seg_neg)
                sentence_batch = Variable(sentence_batch)
                vids = [prefix.split('/')[-1] for prefix in video_prefixes]
                batch_gold_seg = [list(gold_seg[vid].keys()) for vid in vids]
                bert_split = get_matched_recipe_feat(vids, duration_frame)
                if args.cuda:
                    img_batch = img_batch.cuda()
                    tempo_seg_neg = tempo_seg_neg.cuda()
                    tempo_seg_pos = tempo_seg_pos.cuda()
                    sentence_batch = sentence_batch.cuda()
                    recipe_vector = recipe_vector.cuda()
                    neg_frame_flag = neg_frame_flag.cuda()
                    bert_split = bert_split.cuda()
                (pred_score, gt_score,
                 pred_offsets, gt_offsets,
                 pred_sentence, gt_sent,
                 _, mask_loss,_) = model(img_batch, tempo_seg_pos,
                                        tempo_seg_neg, sentence_batch,
                                        stride_factor=args.stride_factor,
                                        gated_mask=args.gated_mask, 
                                       using_recipe = args.using_recipe, recipe_vector = bert_split,
                                                                        vids = vids, gold_segs = batch_gold_seg)

                cls_loss = model.bce_loss(pred_score + 1e-10, gt_score + 1e-10) * args.cls_weight
                reg_loss = model.reg_loss(pred_offsets + 1e-10, gt_offsets + 1e-10) * args.reg_weight
                sent_loss = F.cross_entropy(pred_sentence + 1e-10 , gt_sent) * args.sent_weight

                total_loss = cls_loss + reg_loss + sent_loss #+ attention_loss*0.1+ temporal_loss*0.1

                if mask_loss is not None:
                    mask_loss = args.mask_weight * mask_loss
                    total_loss += mask_loss
                else:
                    mask_loss = cls_loss.new(1).fill_(0)

                valid_loss.append(total_loss.data.item())
                val_cls_loss.append(cls_loss.data.item())
                val_reg_loss.append(reg_loss.data.item())
                val_sent_loss.append(sent_loss.data.item())
                val_mask_loss.append(mask_loss.data.item())
    writer.add_scalar('valid/cls_loss', np.mean(val_cls_loss),epoch)
    writer.add_scalar('valid/sent_loss', np.mean(val_sent_loss),epoch)
    writer.add_scalar('valid/reg_loss', np.mean(val_reg_loss),epoch)
    writer.add_scalar('valid/total_loss', np.mean(valid_loss),epoch)
    #torch.save(model.state_dict(), os.path.join(model_name, 'epoch'+str(epoch)+'.pth'))
    if np.mean(valid_loss)<best_loss:
        torch.save(model.state_dict(), os.path.join(model_name, 'best_eval_loss.pth'))
        best_loss = np.mean(valid_loss)

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


iter: [581/582], training loss: 1.2345, class: 0.0441, reg: 0.1054, sentence: 1.0824, mask: 0.0026, data time: 274.2555s, total time: 274.6373s


In [20]:
len(data)

9

In [21]:
model.load_state_dict(torch.load(os.path.join(model_name, 'best_eval_loss.pth')))

<All keys matched successfully>

In [22]:
### For evaluation captioning to tensorboard
import shlex, subprocess
def get_caption_scores(target):
    scores = {}

    flag = False;
    meteor = None
    Bleu_1 = None
    Bleu_2 = None
    Bleu_3 = None
    Bleu_4 = None
    CIDEr = None
    ROUGE_L = None
    
    


    for line in target.splitlines():    
        if("Average across all tIoUs" in line): flag = True; 
        elif("tIoU" in line): flag = False

        if(flag and "METEOR" in line):
            metric = str(line.split(":")[0].split()[1])
            score = float(line.split(":")[1])
            
            scores[metric] = score
        elif(flag and "Bleu_1" in line):
            metric = str(line.split(":")[0].split()[1])
            score = float(line.split(":")[1])
            
            scores[metric] = score
        elif(flag and "Bleu_2" in line):
            metric = str(line.split(":")[0].split()[1])
            score = float(line.split(":")[1])
            
            scores[metric] = score
        elif(flag and "Bleu_3" in line):
            metric = str(line.split(":")[0].split()[1])
            score = float(line.split(":")[1])
            
            scores[metric] = score
        elif(flag and "Bleu_4" in line):
            metric = str(line.split(":")[0].split()[1])
            score = float(line.split(":")[1])
            
            scores[metric] = score
        elif(flag and "CIDEr" in line):
            metric = str(line.split(":")[0].split()[1])
            score = float(line.split(":")[1])
            
            scores[metric] = score
        elif(flag and "ROUGE_L" in line):
            metric = str(line.split(":")[0].split()[1])
            score = float(line.split(":")[1])
            
            scores[metric] = score
        elif(flag and "Recall" in line):
            metric = str(line.split(":")[0].split()[1])
            score = float(line.split(":")[1])
            
            scores[metric] = score
        elif(flag and "Precision" in line):
            metric = str(line.split(":")[0].split()[1])
            score = float(line.split(":")[1])
            
            scores[metric] = score


    
    
    return scores

In [23]:
    with autocast():
        densecap_result = {}
        for data in tqdm(test_loader):
                (img_batch, tempo_seg_pos, tempo_seg_neg, sentence_batch, query, _, recipe_vector, neg_frame_flag, video_prefixes) = data
                with torch.no_grad():
                    img_batch = Variable(img_batch)
                    tempo_seg_pos = Variable(tempo_seg_pos)
                    tempo_seg_neg = Variable(tempo_seg_neg)
                    sentence_batch = Variable(sentence_batch)
                    vids = [prefix.split('/')[-1] for prefix in video_prefixes]
                    #batch_gold_seg = [list(gold_seg[vid].keys()) for vid in vids]
                    bert_split = get_matched_recipe_feat(vids, duration_frame)
                    original_num_frame = torch.tensor([newof[prefix] for prefix in video_prefixes])
                    if args.cuda:
                        img_batch = img_batch.cuda()
                        tempo_seg_neg = tempo_seg_neg.cuda()
                        tempo_seg_pos = tempo_seg_pos.cuda()
                        sentence_batch = sentence_batch.cuda()
                        recipe_vector = recipe_vector.cuda()
                        neg_frame_flag = neg_frame_flag.cuda()
                        bert_split = bert_split.cuda()
                    all_proposal_results = model.inference(img_batch,
                                                           original_num_frame,
                                                           sampling_sec,
                                                           args.min_prop_num,
                                                           args.max_prop_num,
                                                           args.min_prop_before_nms,
                                                           args.pos_thresh,
                                                           args.stride_factor,
                                                           gated_mask=args.gated_mask, \
                                                           using_recipe=args.using_recipe, recipe_vector = bert_split, dual=None)
                    for b in range(len(video_prefixes)):
                        vid = video_prefixes[b].split('/')[-1]
                        densecap_result['v_'+vid] = []
                        for pred_start, pred_end, pred_s, sent in all_proposal_results[b]:
                            densecap_result['v_'+vid].append(
                                {'sentence':sent,
                                 'timestamp':[pred_start * sampling_sec,
                                              pred_end * sampling_sec]})

        ### Evaluation Captioning Start ....
        ### From all collected captions
        dense_cap_all = {'version':'VERSION 1.0', 'results':densecap_result,
                         'external_data':{'used':'true',
                          'details':'global_pool layer from BN-Inception pretrained from ActivityNet \
                                     and ImageNet (https://github.com/yjxiong/anet2016-cuhk)'}}
        with open(os.path.join('../results/', 'densecap_'+args.val_data_folder[0]+'_'+args.id+ '.json'), 'w') as f:
            json.dump(dense_cap_all, f)
        print("Run eval")
        cmd = 'python3 ' + args.densecap_eval_file + " -s " + os.path.join('/workspace/dense_cap/results/', 'densecap_'+args.val_data_folder[0]+'_' + args.id + '.json') + " -i " + args.id + " -v -r " + args.densecap_references[0]
        #!{cmd}   
        split_cmd = shlex.split(cmd)

        result = subprocess.run(split_cmd, stdout=subprocess.PIPE)
        scores = get_caption_scores(result.stdout.decode())    


Run eval


In [24]:
scores

{'Bleu_1': 23.5795,
 'Bleu_2': 10.6779,
 'Bleu_3': 3.5605,
 'Bleu_4': 1.2511,
 'METEOR': 8.3521,
 'ROUGE_L': 23.2965,
 'CIDEr': 24.0902,
 'Recall': 71.1737,
 'Precision': 16.5019}

In [24]:
scores

{'Bleu_1': 23.3486,
 'Bleu_2': 10.3056,
 'Bleu_3': 3.6186,
 'Bleu_4': 1.4234,
 'METEOR': 8.2698,
 'ROUGE_L': 22.8492,
 'CIDEr': 22.6765,
 'Recall': 67.3056,
 'Precision': 16.5031}

In [24]:
scores

{'Bleu_1': 23.7001,
 'Bleu_2': 10.9155,
 'Bleu_3': 3.846,
 'Bleu_4': 1.6525,
 'METEOR': 8.633,
 'ROUGE_L': 23.3317,
 'CIDEr': 24.0875,
 'Recall': 66.5879,
 'Precision': 16.8477}

In [24]:
scores

{'Bleu_1': 23.0672,
 'Bleu_2': 10.2312,
 'Bleu_3': 3.3494,
 'Bleu_4': 1.2256,
 'METEOR': 8.1385,
 'ROUGE_L': 22.7072,
 'CIDEr': 21.8352,
 'Recall': 67.5833,
 'Precision': 17.7089}

In [24]:
scores

{'Bleu_1': 23.2913,
 'Bleu_2': 10.191,
 'Bleu_3': 3.559,
 'Bleu_4': 1.3747,
 'METEOR': 8.2352,
 'ROUGE_L': 22.8104,
 'CIDEr': 22.6296,
 'Recall': 66.1431,
 'Precision': 17.9357}

In [24]:
scores

{'Bleu_1': 22.9531,
 'Bleu_2': 10.4398,
 'Bleu_3': 3.7087,
 'Bleu_4': 1.365,
 'METEOR': 8.4421,
 'ROUGE_L': 22.9001,
 'CIDEr': 23.4857,
 'Recall': 63.6018,
 'Precision': 15.5384}

In [24]:
scores

{'Bleu_1': 23.357,
 'Bleu_2': 10.4014,
 'Bleu_3': 3.7375,
 'Bleu_4': 1.4228,
 'METEOR': 8.3616,
 'ROUGE_L': 22.948,
 'CIDEr': 23.2781,
 'Recall': 65.0779,
 'Precision': 16.6256}

In [24]:
scores

{'Bleu_1': 22.3137,
 'Bleu_2': 10.1058,
 'Bleu_3': 3.6658,
 'Bleu_4': 1.4506,
 'METEOR': 7.9614,
 'ROUGE_L': 22.2435,
 'CIDEr': 24.4294,
 'Recall': 69.5393,
 'Precision': 15.7502}

In [24]:
scores

{'Bleu_1': 23.2996,
 'Bleu_2': 10.4698,
 'Bleu_3': 3.3981,
 'Bleu_4': 1.2214,
 'METEOR': 8.4136,
 'ROUGE_L': 22.9456,
 'CIDEr': 23.7607,
 'Recall': 69.3233,
 'Precision': 16.6847}

In [24]:
scores

{'Bleu_1': 23.0978,
 'Bleu_2': 10.6308,
 'Bleu_3': 3.82,
 'Bleu_4': 1.4222,
 'METEOR': 8.4893,
 'ROUGE_L': 22.9447,
 'CIDEr': 25.4961,
 'Recall': 68.0257,
 'Precision': 16.2369}

In [7]:
cmd = 'python3 ' + args.densecap_eval_file + " -s " + os.path.join('/workspace/dense_cap/results/', 'densecap_'+args.val_data_folder[0]+'_' + args.id + '.json') + " -i " + args.id + " -v -r " + args.densecap_references[0]

In [8]:
cmd

'python3 /workspace/dense_cap/densecap/tools/densevid_eval/evaluate.py -s /workspace/dense_cap/results/densecap_new_validation_lxmert_sequence_final.json -i lxmert_sequence_final -v -r /workspace/dense_cap/densecap/data/our/new_test_our_yc2_only.json'

In [31]:
with open(os.path.join('../results/', 'densecap_'+args.val_data_folder[0]+'_'+args.id+ '.json'), 'w') as f:
            json.dump(dense_cap_all, f)

In [25]:
scores

{'Bleu_1': 21.5608,
 'Bleu_2': 9.3353,
 'Bleu_3': 3.1344,
 'Bleu_4': 1.2042,
 'METEOR': 7.3952,
 'ROUGE_L': 21.5712,
 'CIDEr': 20.3689,
 'Recall': 67.6871,
 'Precision': 16.7405}

In [24]:
scores

{'Bleu_1': 22.7195,
 'Bleu_2': 10.0923,
 'Bleu_3': 3.7048,
 'Bleu_4': 1.4025,
 'METEOR': 8.1606,
 'ROUGE_L': 22.3009,
 'CIDEr': 23.4594,
 'Recall': 67.2579,
 'Precision': 16.2901}